# DATA2001 Assignment

#First, we will begin by importing the various libraries.

## Importing the libraries
import pandas as pd
import matplotlib as plot
import numpy as np
import geopandas as gpd

In [1]:
from __future__ import (absolute_import, division, print_function)
import os
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, MultiPolygon
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras



In [2]:
data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
        return db,conn

credfilepath = os.path.join(data_path, "data2x01_db.json")
db, conn = pgconnect(credfilepath) #connect to pgAdmin

print(db.table_names())
print("I'm not working right now.")

unable to connect to the database
(psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "soitpw11d59.shared.sydney.edu.au" (10.87.14.236) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/13/e3q8)


UnboundLocalError: local variable 'conn' referenced before assignment

Now, let's import the shape file

In [ ]:
countries = gpd.read_file( os.path.join("/Users/sidath/Desktop/data2001-assignment/RFSNSW_BFPL", "RFSNSW_BFPL.shp") )

print(countries)

Now let's connect this to a database.

In [ ]:
countries = countries.dropna()
print(countries)

In [ ]:
def pgexecute( conn, sqlcmd, args=None, msg='', silent=False ):
    """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
    retval = False
    result_set = None

    try:
        if args is None:
            result_set = conn.execute(sqlcmd).fetchall()
        else:
            result_set = conn.execute(sqlcmd, args).fetchall()

        if silent == False: 
            print("success: " + msg)
            for record in result_set:
                print(record)
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval

def pgquery( conn, sqlcmd, args=None, silent=False ):
    """ utility function to execute some SQL query statement
    can take optional arguments to fill in (dictionary)
    will print out on screen the result set of the query
    error and transaction handling built-in """
    retdf = pd.DataFrame()
    retval = False
    try:
        if args is None:
            retdf = pd.read_sql_query(sqlcmd,conn)
        else:
            retdf = pd.read_sql_query(sqlcmd,conn,params=args)
        if silent == False:
            print(retdf.shape)
            print(retdf.to_string())
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval,retdf

In [ ]:
# lets just make sure we have PostGIS working on our connection
postgis_check = '''
SELECT PostGIS_Version();
'''

retval,retdf = pgquery(conn,postgis_check)
retdf

In [ ]:
RFSNSW_BFPL = '''CREATE TABLE rfsnsw_bfpl (
                     "CATEGORY" NUMERIC, 
                     "SHAPE_LENG" NUMERIC, 
                     "SHAPE_AREA" VARCHAR(50), 
                     location GEOMETRY(POINT,4326))''' 

conn.execute("DROP TABLE IF EXISTS rfsnsw_bfpl")
conn.execute(RFSNSW_BFPL)

In [3]:
# Use GeoAlchemy's WKTElement to create a geom with SRID
# NOTE :: This time we are assuming everything is a point so only wkt conversion is required.
def create_wkt_point_element(geom,srid):
    return WKTElement(geom.wkt, srid)




In [4]:
srid = 4326

# since we are altering data again, we should create another copy
citiesWkCpy = countries.copy()
citiesWkCpy['location'] = citiesWkCpy['geometry'].apply(lambda x: create_wkt_point_element(geom=x,srid=srid))
#delete the old column before insert
citiesWkCpy = citiesWkCpy.drop(columns="geometry")
citiesWkCpy



NameError: name 'countries' is not defined

In [5]:
# Then insert the data from the GeoPandas DataFrame to PostGIS Table
# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
cities_table_name = "rfsnsw_bfpl"
citiesWkCpy.to_sql(cities_table_name, conn, if_exists='append', index=False, 
                         dtype={'location': Geometry('POINT', srid)})

NameError: name 'citiesWkCpy' is not defined

In [6]:
query = "SELECT COUNT(*) FROM rfsnsw_bfpl"
retval,retdf = pgquery(conn, query)
retdf

NameError: name 'pgquery' is not defined